In [1]:
# For local VSCODE and Jupyter ONLY:

import os
from litellm import completion
from dotenv import load_dotenv
from typing import List, Dict


load_dotenv()

HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")
OLLAMA_API_KEY = os.getenv("OLLAMA_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

# print(HUGGINGFACE_API_KEY)
# print(OLLAMA_API_KEY)
# print(GEMINI_API_KEY)

API_KEY = GEMINI_API_KEY # HUGGINGFACE_API_KEY OR OLLAMA_API_KEY # OR GEMINI_API_KEY

In [8]:
from litellm import completion
from typing import List, Dict
import json

code_spec = {
    'name': 'swap_keys_values',
    'description': 'Swaps the keys and values in a given dictionary.',
    'params': {
        'd': 'A dictionary with unique values.'
    },
}


def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get response"""
    response = completion(
        model="gemini/gemini-2.0-flash", 
        messages=messages,
        max_tokens=1024,
        api_key=API_KEY
    )
    return response.choices[0].message.content

prompt1 = input("What do you want to create?: ")

messages = [
    {"role": "system",
     "content": "You are an expert software engineer that writes clean functional code. You do not document your code, but you do write clean and functional code. You are given a task to complete by writing only a basic python function based on the desccription provided by the user."},
    {"role": "user", "content": prompt1},
]


response1 = generate_response(messages)
print("Generated Code:\n", response1)

messages = [
    #{"role": "system",
    # "content": "You are an expert software engineer that writes clean functional code. You do not document your code, but you do write clean and functional code. You are given a task to complete by writing only a basic python function based on the desccription provided by the user."},
    {"role": "user", "content": prompt1},
    {"role": "assistant", "content": response1},
    {"role": "system", "content": """Add comprehensive documentation including:
        Function description
        Parameter descriptions
        Return value description
        Example usage
        Edge cases.
        to the code provided by the assistant."""},
]

response2 = generate_response(messages)
print("Generated Documentation:\n", response2)

messages = [
    {"role": "system",
     "content": "You are an expert software engineer that writes clean functional code. You do not document your code, but you do write clean and functional code. You are given a task to complete by writing only a basic python function based on the desccription provided by the user."},
    {"role": "user", "content": prompt1},
    {"role": "assistant", "content": response1},
    {"role": "system", "content": """Add comprehensive documentation including:
        Function description
        Parameter descriptions
        Return value description
        Example usage
        Edge cases.
        to the code provided by the assistant."""},
    {"role": "assistant", "content": response2}, 
    {"role": "system", "content":  """add test cases using Python's unittest framework
        Tests should cover:
        Basic functionality
        Edge cases
        Error cases
        Various input scenarios"""}
]

response3 = generate_response(messages)
print("Generated Test Cases:\n", response3)


Generated Code:
 ```python
def bubblesort(list):
    n = len(list)
    for i in range(n):
        for j in range(0, n-i-1):
            if list[j] > list[j+1]:
                list[j], list[j+1] = list[j+1], list[j]
```

Generated Documentation:
 None
Generated Test Cases:
 ```python
def bubblesort(list):
    """
    Sorts a list of elements in ascending order using the bubble sort algorithm.

    The bubble sort algorithm repeatedly steps through the list, compares adjacent elements,
    and swaps them if they are in the wrong order. The pass through the list is repeated
    until no swaps are needed, which indicates that the list is sorted.

    Parameters:
    list (list): A list of comparable elements (e.g., integers, strings) to be sorted.
                 The list is sorted in-place.

    Returns:
    list: The sorted list. The original list is modified.

    Example Usage:
    >>> my_list = [5, 1, 4, 2, 8]
    >>> bubblesort(my_list)
    [1, 2, 4, 5, 8]

    >>> my_list = [5, 1,

In [10]:
###### Thier solution ######

from litellm import completion
from typing import List, Dict
import sys

def generate_response(messages: List[Dict]) -> str:
   """Call LLM to get response"""
   response = completion(
      model="gemini/gemini-2.0-flash", 
      messages=messages,
      max_tokens=1024,
      api_key=API_KEY
   )
   return response.choices[0].message.content

def extract_code_block(response: str) -> str:
   """Extract code block from response"""

   if not '```' in response:
      return response

   code_block = response.split('```')[1].strip()
   # Check for "python" at the start and remove

   if code_block.startswith("python"):
      code_block = code_block[6:]

   return code_block

def develop_custom_function():
   # Get user input for function description
   print("\nWhat kind of function would you like to create?")
   print("Example: 'A function that calculates the factorial of a number'")
   print("Your description: ", end='')
   function_description = input().strip()

   # Initialize conversation with system prompt
   messages = [
      {"role": "system", "content": "You are a Python expert helping to develop a function."}
   ]

   # First prompt - Basic function
   messages.append({
      "role": "user",
      "content": f"Write a Python function that {function_description}. Output the function in a ```python code block```."
   })
   initial_function = generate_response(messages)

   # Parse the response to get the function code
   initial_function = extract_code_block(initial_function)

   print("\n=== Initial Function ===")
   print(initial_function)

   # Add assistant's response to conversation
   # Notice that I am purposely causing it to forget its commentary and just see the code so that
   # it appears that is always outputting just code.
   messages.append({"role": "assistant", "content": "\`\`\`python\n\n"+initial_function+"\n\n\`\`\`"})

   # Second prompt - Add documentation
   messages.append({
      "role": "user",
      "content": "Add comprehensive documentation to this function, including description, parameters, "
                 "return value, examples, and edge cases. Output the function in a ```python code block```."
   })
   documented_function = generate_response(messages)
   documented_function = extract_code_block(documented_function)
   print("\n=== Documented Function ===")
   print(documented_function)

   # Add documentation response to conversation
   messages.append({"role": "assistant", "content": "\`\`\`python\n\n"+documented_function+"\n\n\`\`\`"})

   # Third prompt - Add test cases
   messages.append({
      "role": "user",
      "content": "Add unittest test cases for this function, including tests for basic functionality, "
                 "edge cases, error cases, and various input scenarios. Output the code in a \`\`\`python code block\`\`\`."
   })
   test_cases = generate_response(messages)
   # We will likely run into random problems here depending on if it outputs JUST the test cases or the
   # test cases AND the code. This is the type of issue we will learn to work through with agents in the course.
   test_cases = extract_code_block(test_cases)
   print("\n=== Test Cases ===")
   print(test_cases)

   # Generate filename from function description
   filename = function_description.lower()
   filename = ''.join(c for c in filename if c.isalnum() or c.isspace())
   filename = filename.replace(' ', '_')[:30] + '.py'

   # Save final version
   with open(filename, 'w') as f:
      f.write(documented_function + '\n\n' + test_cases)

   return documented_function, test_cases, filename

if __name__ == "__main__":


   function_code, tests, filename = develop_custom_function()
   print(f"\nFinal code has been saved to {filename}")

<>:59: SyntaxWarning: invalid escape sequence '\`'
<>:59: SyntaxWarning: invalid escape sequence '\`'
<>:73: SyntaxWarning: invalid escape sequence '\`'
<>:73: SyntaxWarning: invalid escape sequence '\`'
<>:79: SyntaxWarning: invalid escape sequence '\`'
<>:59: SyntaxWarning: invalid escape sequence '\`'
<>:59: SyntaxWarning: invalid escape sequence '\`'
<>:73: SyntaxWarning: invalid escape sequence '\`'
<>:73: SyntaxWarning: invalid escape sequence '\`'
<>:79: SyntaxWarning: invalid escape sequence '\`'
/var/folders/nk/rwl926752110m9s2n5nj8cl40000gn/T/ipykernel_13968/654272443.py:59: SyntaxWarning: invalid escape sequence '\`'
  messages.append({"role": "assistant", "content": "\`\`\`python\n\n"+initial_function+"\n\n\`\`\`"})
/var/folders/nk/rwl926752110m9s2n5nj8cl40000gn/T/ipykernel_13968/654272443.py:59: SyntaxWarning: invalid escape sequence '\`'
  messages.append({"role": "assistant", "content": "\`\`\`python\n\n"+initial_function+"\n\n\`\`\`"})
/var/folders/nk/rwl926752110m9s2n5


What kind of function would you like to create?
Example: 'A function that calculates the factorial of a number'
Your description: 
=== Initial Function ===

def bubble_sort(arr):
  """
  Sorts a list of numbers in ascending order using the bubble sort algorithm.

  Args:
    arr: The list of numbers to sort.

  Returns:
    The sorted list.
  """
  n = len(arr)
  for i in range(n):
    # Flag to optimize, if no swaps occur, the list is sorted
    swapped = False
    for j in range(0, n - i - 1):
      if arr[j] > arr[j + 1]:
        arr[j], arr[j + 1] = arr[j + 1], arr[j]
        swapped = True
    if not swapped:
      break  # If no two elements were swapped in inner loop, the array is sorted
  return arr

=== Documented Function ===

def bubble_sort(arr):
  """
  Sorts a list of numbers in ascending order using the bubble sort algorithm.

  The bubble sort algorithm repeatedly steps through the list, compares adjacent
  elements, and swaps them if they are in the wrong order. The p